# Assignment 2 for Research Track 2

The goal of this challenge is to build a Jupyter Notebook that serves as the controller for a robot that operates in a specific area. Users will primarily be able to communicate with this notebook through its User Interface, which will be made up of the following elements:

1. Control Buttons: These buttons are used to direct how the robot moves about its surroundings.

2. Position and Target Plot: This diagram shows where the robot and its targets are in relation to their surroundings.

3. Distance Display: A text box will display the distance to the nearest barrier.

4. The Target Achievement Plot compares the number of goals the robot has achieved against the number of goals it has not yet attained.

In [ ]:
import rospy
import actionlib
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
from geometry_msgs.msg import PoseStamped 
import actionlib.msg 
from sensor_msgs.msg import LaserScan
import ipywidgets as widgets
import assignment_2_2022.msg 
import os
from IPython.display import display
import matplotlib.pyplot as plt
from nav_msgs.msg import Odometry
target_reached = 0
target_canceled = 0
%matplotlib widget

In [ ]:
def send_target(x, y):
    rospy.init_node('jupyter_interface')
    client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)  
    client.wait_for_server()

    x_pose = int(x)
    y_pose = int(y)
    target = PoseStamped()


    target.pose.position.x = x_pose
    target.pose.position.y = y_pose


    target = assignment_2_2022.msg.PlanningGoal(target)    
    client.send_goal(target)
    client.wait_for_result()

def set_target_coordinates(button):
    x = int(x_text.value)
    y = int(y_text.value)
    send_target(x, y)

# Create the button and input fields
x_text = widgets.Text(description='X:')
y_text = widgets.Text(description='Y:')
button = widgets.Button(description='send target')

# Attach the button callback function
button.on_click(set_target_coordinates)

# Display the widgets
display(x_text, y_text, button)

In [ ]:
plt.switch_backend('TKAgg')

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.goal_ln, = plt.plot([], [], 'go') 
        self.x_data, self.y_data = [], []
        self.goal_x, self.goal_y = None, None

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln, self.goal_ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def goal_callback(self, msg):
        self.goal_x = msg.x
        self.goal_y = msg.y

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.goal_ln.set_data(self.goal_x, self.goal_y)
        return self.ln, self.goal_ln


vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
des_pos_x = rospy.get_param("/des_pos_x")
des_pos_y = rospy.get_param("/des_pos_y")
vis.goal_x = des_pos_x
vis.goal_y = des_pos_y
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)


In [ ]:
class obstacle:
    def __init__(self):
        self.distance_text = widgets.FloatText(description='distance from closest obstacle:', value=5)

    def laser_callback(self, msg):
        closest_distance = min(msg.ranges)
        self.distance_text.value = closest_distance

vis = obstacle()
sub = rospy.Subscriber('/scan', LaserScan, vis.laser_callback)

display(vis.distance_text)

In [ ]:
plt.switch_backend('TKAgg')
def callback_subscriber(msg):
    global target_reached, target_canceled
    
    if msg.status.status == 2:
        target_canceled += 1
    elif msg.status.status == 3:
        target_reached += 1

rospy.Subscriber("/reaching_goal/result", assignment_2_2022.msg.PlanningActionResult, callback_subscriber)
x_data = ['Reached Goals', 'Canceled Goals']
y_data = [target_reached, target_canceled]
plt.bar(x_data, y_data)
plt.xlabel('Goal Status')
plt.ylabel('Count')
plt.title('Reached and Canceled Goals')
plt.show()
